In [ ]:
import joblib
import pandas as pd
import numpy as np
# import nltk
# nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize.regexp import regexp_tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

In [ ]:
import sys

sys.executable

'C:\\Users\\ajaco\\Anaconda3\\python.exe'

In [ ]:
sys.path

['C:\\Users\\ajaco',
 'C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\deploying-machine-learning-models\\packages\\regression_model',
 'C:\\Users\\ajaco\\Anaconda3\\python37.zip',
 'C:\\Users\\ajaco\\Anaconda3\\DLLs',
 'C:\\Users\\ajaco\\Anaconda3\\lib',
 'C:\\Users\\ajaco\\Anaconda3',
 '',
 'C:\\Users\\ajaco\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\ajaco\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\ajaco\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\ajaco\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\ajaco\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\ajaco\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\ajaco\\.ipython']

In [ ]:
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\ajaco\Anaconda3\share\jupyter\kernels\python3


In [3]:
df = pd.read_csv('twelve_twelve_update.csv')

print(df.shape)

df.head()

(11114, 15)


,From,Message,Subject,Tags,UID,text,tag_list,first_tag,sender_name,sender_email,domain_name,isNoReply,link_count,clean_text,is_personal
0,<grangepayments@westernunionspeedpay.com>,"Dear AVRAHAM JACOBSOHN, This is to confirm th...",Grange Payment Confirmation,Finance,31780,<grangepayments@westernunionspeedpay.com> Gran...,['Finance'],Finance,NaN,grangepayments@westernunionspeedpay.com,westernunionspeedpay,False,0,Dear AVRAHAM JACOBSOHN This is to confirm that...,False
1,Chase <no.reply.alerts@chase.com>,This is an Alert to help manage your account ...,Your Debit Card Transaction,Finance,31779,Chase <no.reply.alerts@chase.com> Your Debit C...,['Finance'],Finance,Chase,no.reply.alerts@chase.com,chase,True,0,This is an Alert to help manage your account e...,False
2,Amazon Web Services <no-reply-aws@amazon.com>,Please let us know if we helped resolve your i...,Resolved 6559329691: Limit Increase: SageMaker,Productivity,31738,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True,3,Please let us know if we helped resolve your i...,False
3,Lambda Labs <noreply@github.com>,Youve been added to the Labs 18 - Tagger team ...,Bernie Durfee added you to the Lambda Labs tea...,Productivity,31693,Lambda Labs <noreply@github.com> Bernie Durfee...,['Productivity'],Productivity,Lambda Labs,noreply@github.com,github,True,2,Youve been added to the Labs Tagger team for t...,False
4,Amazon Web Services <no-reply-aws@amazon.com>,"Hello, We haven't heard back from you regard...",Attention required on case 6559329691: Limit I...,Productivity,31684,Amazon Web Services <no-reply-aws@amazon.com> ...,['Productivity'],Productivity,Amazon Web Services,no-reply-aws@amazon.com,amazon,True,2,Hello We haven't heard back from you regarding...,False


In [4]:
train, test = train_test_split(df, stratify=df['is_personal'])

train.shape, test.shape

((8335, 15), (2779, 15))

In [14]:
features = 'clean_text'
target = 'is_personal'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8335,), (8335,), (2779,), (2779,))

In [ ]:
def regnltk_tokenize(text, lemmatizer, tokenizer):
    words = tokenizer(text, pattern='\s+', gaps=True)
    return [lemmatizer.lemmatize(word) for word in words if (len(word) >= 3)]

In [ ]:
def regnltk_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    words = regexp_tokenize(text, pattern='\s+', gaps=True)
    return [lemmatizer.lemmatize(word) for word in words if (len(word) >= 3)]

In [ ]:
vect = joblib.load('C://Users/ajaco/Downloads/vect.joblib')

C:\Users\ajaco\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\ajaco\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

C:\Users\ajaco\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'le', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [8]:
model = LogisticRegression(solver='lbfgs')

model.fit(X_train, y_train)

print('Logistic Regression Accuracy: ', model.score(X_test, y_test))

Logistic Regression Accuracy:  0.9618567830154732


#### This section I try to recreate 97% accuracy with basic TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [ ]:
model.fit(X_train, y_train)

# 96.3% acc. 
print('Logistic Regression Accuracy: ', model.score(X_test, y_test))

Logistic Regression Accuracy:  0.9636559913638


### Here would like to try gensim or spacy tokenizer

In [ ]:
my_stopwords = STOPWORDS.union(set(['jacobsohn', 'avraham', 'http', 'https', 'kalman', 'com', 'sdui', 'www']))

In [ ]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in my_stopwords]

In [22]:
vect = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1,1), min_df=0.05, max_df=0.95)
model = LogisticRegression(solver='lbfgs')

features = 'clean_text'
target = 'is_personal'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

model.fit(X_train, y_train)

print('Logistic Regression Accuracy: ', model.score(X_test, y_test))

Logistic Regression Accuracy:  0.9453040662108673


#### Here would like to try a grid search if that even makes sense.

In [10]:
joblib.dump(model, 'logreg.pkl')

['logreg.pkl']

In [12]:
joblib.dump(tfidf, 'basic_tfidf.pkl')

['basic_tfidf.pkl']